# Tutorial 04: computation of the inf-sup constant for a Stokes problem discretization

In this tutorial we compare the computation of the inf-sup constant of a Stokes problem using standard assembly with mixed function spaces and block assembly.

In [ ]:
import numpy as np
from mpi4py import MPI
from petsc4py import PETSc
from slepc4py import SLEPc
from ufl import (div, dx, FiniteElement, grad, inner, MixedElement, split, TestFunction, TrialFunction,
                 VectorElement)
from dolfinx import Constant, Function, FunctionSpace, UnitSquareMesh
from dolfinx.cpp.fem import create_vector_block
from dolfinx.fem import (assemble_matrix, assemble_matrix_block, assemble_scalar, BlockVecSubVectorWrapper,
                         DofMapRestriction, locate_dofs_topological, VecSubVectorWrapper)
from dolfinx.mesh import locate_entities_geometrical
from dolfinx.plotting import plot

### Mesh

In [ ]:
mesh = UnitSquareMesh(MPI.COMM_WORLD, 32, 32)


def wall(x):
    return np.logical_or(x[1] < 0 + np.finfo(float).eps, x[1] > 1 - np.finfo(float).eps)


boundary_facets = locate_entities_geometrical(mesh, mesh.topology.dim - 1, wall)

In [ ]:
plot(mesh)

### Function spaces

In [ ]:
V_element = VectorElement("Lagrange", mesh.ufl_cell(), 2)
Q_element = FiniteElement("Lagrange", mesh.ufl_cell(), 1)

### Auxiliary function for eigenvector normalization

In [ ]:
def normalize(u1, u2, p):
    u1_norm = np.sqrt(mesh.mpi_comm().allreduce(assemble_scalar(inner(grad(u1), grad(u1)) * dx), op=MPI.SUM))
    u1.vector.scale(1. / u1_norm)
    u1.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
    u2_norm = np.sqrt(mesh.mpi_comm().allreduce(assemble_scalar(inner(grad(u2), grad(u2)) * dx), op=MPI.SUM))
    u2.vector.scale(1. / u2_norm)
    u2.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
    p_norm = np.sqrt(mesh.mpi_comm().allreduce(assemble_scalar(p * p * dx), op=MPI.SUM))
    p.vector.scale(1. / p_norm)
    p.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

### Standard FEniCSx formulation using a mixed function space

In [ ]:
def run_monolithic():
    # Function spaces
    W_element = MixedElement(V_element, Q_element)
    W = FunctionSpace(mesh, W_element)

    # Test and trial functions: monolithic
    vq = TestFunction(W)
    (v, q) = split(vq)
    up = TrialFunction(W)
    (u, p) = split(up)

    # Variational forms
    lhs = inner(grad(u), grad(v)) * dx - div(v) * p * dx - div(u) * q * dx
    rhs = - inner(p, q) * dx

    # Define restriction for DOFs associated to homogenous Dirichlet boundary conditions
    dofs_W = np.arange(0, W.dofmap.index_map.block_size * (
        W.dofmap.index_map.size_local + W.dofmap.index_map.num_ghosts))
    bdofs_V = locate_dofs_topological(W.sub(0), mesh.topology.dim - 1, boundary_facets)
    restriction = DofMapRestriction(W.dofmap, np.setdiff1d(dofs_W, bdofs_V))

    # Assemble lhs and rhs matrices
    A = assemble_matrix(lhs, restriction=(restriction, restriction))
    A.assemble()
    B = assemble_matrix(rhs, restriction=(restriction, restriction))
    B.assemble()

    # Solve
    eps = SLEPc.EPS().create()
    eps.setOperators(A, B)
    eps.setProblemType(SLEPc.EPS.ProblemType.GNHEP)
    eps.setDimensions(1, PETSc.DECIDE, PETSc.DECIDE)
    eps.setWhichEigenpairs(SLEPc.EPS.Which.TARGET_REAL)
    eps.setTarget(1.e-5)
    eps.getST().setType(SLEPc.ST.Type.SINVERT)
    eps.getST().getKSP().setType("preonly")
    eps.getST().getKSP().getPC().setType("lu")
    eps.getST().getKSP().getPC().setFactorSolverType("mumps")
    eps.solve()
    assert eps.getConverged() >= 1

    # Extract leading eigenvalue and eigenvector
    vr = create_vector_block([restriction.index_map])
    vi = create_vector_block([restriction.index_map])
    eigv = eps.getEigenpair(0, vr, vi)
    r, i = eigv.real, eigv.imag
    assert abs(i) < 1.e-10
    assert r > 0., "r = " + str(r) + " is not positive"
    print("Inf-sup constant (monolithic): ", np.sqrt(r))

    # Transform eigenvector into eigenfunction
    r_fun = Function(W)
    vr.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
    with r_fun.vector.localForm() as r_fun_local, VecSubVectorWrapper(vr, W.dofmap, restriction) as vr_wrapper:
        r_fun_local[:] = vr_wrapper
    (u_fun_1, u_fun_2) = (r_fun.sub(0).sub(0).collapse(), r_fun.sub(0).sub(1).collapse())
    p_fun = r_fun.sub(1).collapse()
    normalize(u_fun_1, u_fun_2, p_fun)

    return (r, u_fun_1, u_fun_2, p_fun)

In [ ]:
(eig_m, u_fun_1_m, u_fun_2_m, p_fun_m) = run_monolithic()

In [ ]:
plot(u_fun_1_m)

In [ ]:
plot(u_fun_2_m)

In [ ]:
plot(p_fun_m)

### Block FEniCSx formulation using a two independent function spaces

In [ ]:
def run_block():
    # Function spaces
    V = FunctionSpace(mesh, V_element)
    Q = FunctionSpace(mesh, Q_element)

    # Test and trial functions
    (v, q) = (TestFunction(V), TestFunction(Q))
    (u, p) = (TrialFunction(V), TrialFunction(Q))

    # Variational forms
    lhs = [[inner(grad(u), grad(v)) * dx, - div(v) * p * dx],
           [- div(u) * q * dx, None]]
    rhs = [[None, None],
           [None, - p * q * dx]]
    rhs[0][0] = Constant(mesh, 0.) * inner(u, v) * dx

    # Define restriction for DOFs associated to homogenous Dirichlet boundary conditions
    dofs_V = np.arange(0, V.dofmap.index_map.block_size * (
        V.dofmap.index_map.size_local + V.dofmap.index_map.num_ghosts))
    bdofs_V = locate_dofs_topological(V, mesh.topology.dim - 1, boundary_facets)
    dofs_Q = np.arange(0, Q.dofmap.index_map.block_size * (
        Q.dofmap.index_map.size_local + Q.dofmap.index_map.num_ghosts))
    restriction_V = DofMapRestriction(V.dofmap, np.setdiff1d(dofs_V, bdofs_V))
    restriction_Q = DofMapRestriction(Q.dofmap, dofs_Q)
    restriction = [restriction_V, restriction_Q]

    # Assemble lhs and rhs matrices
    A = assemble_matrix_block(lhs, bcs=[], restriction=(restriction, restriction))
    A.assemble()
    B = assemble_matrix_block(rhs, bcs=[], restriction=(restriction, restriction))
    B.assemble()

    # Solve
    eps = SLEPc.EPS().create()
    eps.setOperators(A, B)
    eps.setProblemType(SLEPc.EPS.ProblemType.GNHEP)
    eps.setDimensions(1, PETSc.DECIDE, PETSc.DECIDE)
    eps.setWhichEigenpairs(SLEPc.EPS.Which.TARGET_REAL)
    eps.setTarget(1.e-5)
    eps.getST().setType(SLEPc.ST.Type.SINVERT)
    eps.getST().getKSP().setType("preonly")
    eps.getST().getKSP().getPC().setType("lu")
    eps.getST().getKSP().getPC().setFactorSolverType("mumps")
    eps.solve()
    assert eps.getConverged() >= 1

    # Extract leading eigenvalue and eigenvector
    vr = create_vector_block([restriction_.index_map for restriction_ in restriction])
    vi = create_vector_block([restriction_.index_map for restriction_ in restriction])
    eigv = eps.getEigenpair(0, vr, vi)
    r, i = eigv.real, eigv.imag
    assert abs(i) < 1.e-10
    assert r > 0., "r = " + str(r) + " is not positive"
    print("Inf-sup constant (block): ", np.sqrt(r))

    # Transform eigenvector into eigenfunction
    (u_fun, p_fun) = (Function(V), Function(Q))
    vr.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
    with BlockVecSubVectorWrapper(vr, [V.dofmap, Q.dofmap], restriction) as vr_wrapper:
        for vr_wrapper_local, component in zip(vr_wrapper, (u_fun, p_fun)):
            with component.vector.localForm() as component_local:
                component_local[:] = vr_wrapper_local
    (u_fun_1, u_fun_2) = (u_fun.sub(0).collapse(), u_fun.sub(1).collapse())
    normalize(u_fun_1, u_fun_2, p_fun)

    return (r, u_fun_1, u_fun_2, p_fun)

In [ ]:
(eig_b, u_fun_1_b, u_fun_2_b, p_fun_b) = run_block()

In [ ]:
plot(u_fun_1_b)

In [ ]:
plot(u_fun_2_b)

In [ ]:
plot(p_fun_b)

### Error computation between FEniCSx and multiphenics

In [ ]:
def run_error(eig_m, eig_b, u_fun_1_m, u_fun_1_b, u_fun_2_m, u_fun_2_b, p_fun_m, p_fun_b):
    err_inf_sup = abs(np.sqrt(eig_b) - np.sqrt(eig_m)) / np.sqrt(eig_m)
    print("Relative error for inf-sup constant equal to", err_inf_sup)
    assert np.isclose(err_inf_sup, 0., atol=1.e-8)
    # Even after normalization, eigenfunctions may have different signs. Try both and assume that the correct
    # error computation is the one for which the error is minimum
    err_1_plus = u_fun_1_b + u_fun_1_m
    err_2_plus = u_fun_2_b + u_fun_2_m
    err_p_plus = p_fun_b + p_fun_m
    err_1_minus = u_fun_1_b - u_fun_1_m
    err_2_minus = u_fun_2_b - u_fun_2_m
    err_p_minus = p_fun_b - p_fun_m
    err_1_plus_norm = np.sqrt(
        mesh.mpi_comm().allreduce(assemble_scalar(inner(grad(err_1_plus), grad(err_1_plus)) * dx)))
    err_2_plus_norm = np.sqrt(
        mesh.mpi_comm().allreduce(assemble_scalar(inner(grad(err_2_plus), grad(err_2_plus)) * dx)))
    err_p_plus_norm = np.sqrt(
        mesh.mpi_comm().allreduce(assemble_scalar(err_p_plus * err_p_plus * dx)))
    err_1_minus_norm = np.sqrt(
        mesh.mpi_comm().allreduce(assemble_scalar(inner(grad(err_1_minus), grad(err_1_minus)) * dx)))
    err_2_minus_norm = np.sqrt(
        mesh.mpi_comm().allreduce(assemble_scalar(inner(grad(err_2_minus), grad(err_2_minus)) * dx)))
    err_p_minus_norm = np.sqrt(
        mesh.mpi_comm().allreduce(assemble_scalar(err_p_minus * err_p_minus * dx)))
    u_fun_1_norm = np.sqrt(
        mesh.mpi_comm().allreduce(assemble_scalar(inner(grad(u_fun_1_m), grad(u_fun_1_m)) * dx)))
    u_fun_2_norm = np.sqrt(
        mesh.mpi_comm().allreduce(assemble_scalar(inner(grad(u_fun_2_m), grad(u_fun_2_m)) * dx)))
    p_fun_norm = np.sqrt(
        mesh.mpi_comm().allreduce(assemble_scalar(p_fun_m * p_fun_m * dx)))

    def select_error(err_plus, err_plus_norm, err_minus, err_minus_norm, vec_norm, component_name):
        ratio_plus = err_plus_norm / vec_norm
        ratio_minus = err_minus_norm / vec_norm
        if ratio_minus < ratio_plus:
            print("Relative error for ", component_name, "component of eigenvector equal to",
                  ratio_minus, "(the one with opposite sign was", ratio_plus, ")")
            assert np.isclose(ratio_minus, 0., atol=1.e-6)
        else:
            print("Relative error for", component_name, "component of eigenvector equal to",
                  ratio_plus, "(the one with opposite sign was", ratio_minus, ")")
            assert np.isclose(ratio_plus, 0., atol=1.e-6)

    select_error(err_1_plus, err_1_plus_norm, err_1_minus, err_1_minus_norm, u_fun_1_norm, "velocity 1")
    select_error(err_2_plus, err_2_plus_norm, err_2_minus, err_2_minus_norm, u_fun_2_norm, "velocity 2")
    select_error(err_p_plus, err_p_plus_norm, err_p_minus, err_p_minus_norm, p_fun_norm, "pressure")

In [ ]:
run_error(eig_m, eig_b, u_fun_1_m, u_fun_1_b, u_fun_2_m, u_fun_2_b, p_fun_m, p_fun_b)